# Update Abstimmungsresultate
Datum: 28.11.2021

Immer nach den Abstimmungen werden die Abstimmungsresultate vom Datamanagement aufbereitet und auf der «[**Abstimmungs-Datenbank
von Statistik Stadt Zürich**](http://sszsttprd/statisticstools/Modules/Abstimmungen/Intranet/)» verfügbar gemacht.

Für OGD kann dann via Exportfunktion (Format Roh > Export) ein Excelfile exportiert und unter folgendem Pfad abgelegt werden: 
`\\szh.loc\ssz\applikationen\OGD\Daten\Quelldaten\SSZ\Politik\abstimmungen\__Modules_Abstimmungen_Export_AbstimmungenExport.xls`

Damit das die Ressourcen des Datasets «[Abstimmungsresultate der Stadt Zürich, seit 1933](https://data.stadt-zuerich.ch/dataset/politik-abstimmungen-seit-1933)» **manuell** aktualisiert werden können, muss das exportierte Excel noch 
 - als **Csv** aufbereitet und
 - das Attribut 'Abstimmungs_Datum' noch ins korrekte Datumsformat (**JJJJ-MM-TT / ISO 8601**) umgewandelt
 werden.
 
Früher wurden diese simplen Schritte im Libre Office (Open Office) gemacht. Nun sollen sie noch einfacher per Python gemacht werden.



<!---  
Dieses Notebook verwendet als Vorlage die Webseite «[Kitas in Berlin - Jupyter Notebook](https://juanitorduz.github.io/kitas_berlin/)» 
---!>


In [1]:
#%pip install geopandas fiona requests folium mplleaflet contextily

In [2]:
import numpy as np
import pandas as pd
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui


## Zuletzt veröffentlichte Abstimmungsresultate
Wie sehen die aktuell auf dem OGD-Katalog veröffentlichten Daten dazu aus?

In [3]:
url_abstimmungsresultate =r"https://data.stadt-zuerich.ch/dataset/politik-abstimmungen-seit-1933/download/abstimmungen_seit1933.csv"

# Read the data
bisherige_abstimmungsresultate = pd.read_csv(
    url_abstimmungsresultate, 
    sep=','    
    )
bisherige_abstimmungsresultate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9708 entries, 0 to 9707
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Abstimmungs_Datum      9708 non-null   object 
 1   Nr_Politische_Ebene    9708 non-null   int64  
 2   Name_Politische_Ebene  9708 non-null   object 
 3   Abstimmungs_Text       9708 non-null   object 
 4   Nr_Resultat_Gebiet     9708 non-null   int64  
 5   Name_Resultat_Gebiet   9708 non-null   object 
 6   Nr_Wahlkreis_StZH      5140 non-null   float64
 7   Name_Wahlkreis_StZH    5140 non-null   object 
 8   Stimmberechtigt        6573 non-null   float64
 9   Ja                     9708 non-null   int64  
 10  Nein                   9708 non-null   int64  
 11  Stimmbeteiligung (%)   9708 non-null   float64
 12  Ja (%)                 9708 non-null   float64
 13  Nein (%)               9708 non-null   float64
 14  StaendeJa              382 non-null    object 
 15  Stae

In [4]:
bisherige_abstimmungsresultate.head(2)

,Abstimmungs_Datum,Nr_Politische_Ebene,Name_Politische_Ebene,Abstimmungs_Text,Nr_Resultat_Gebiet,Name_Resultat_Gebiet,Nr_Wahlkreis_StZH,Name_Wahlkreis_StZH,Stimmberechtigt,Ja,Nein,Stimmbeteiligung (%),Ja (%),Nein (%),StaendeJa,StaendeNein
0,2021-09-26,1,Eidgenossenschaft,"Volksinitiative «Löhne entlasten, Kapital gere...",1,Eidgenossenschaft,NaN,NaN,NaN,986901,1823056,52.2,35.1,64.9,0,20 6/2
1,2021-09-26,1,Eidgenossenschaft,"Volksinitiative «Löhne entlasten, Kapital gere...",2,Kanton Zürich,NaN,NaN,955494.0,182856,325621,54.5,36.0,64.0,NaN,NaN


## Importiere die neusten Abstimmungsresultate zur Datenaufbereitung

Wir importieren das von der Abstimmungsdatenbank exportierte Excelfile und bereiten es für die Veröffentlichung für OGD auf.


Link zu `read_excel()` in Pandas: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html

`pandas.read_excel(io, sheet_name=0, header=0, names=None, index_col=None, usecols=None, squeeze=False, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, parse_dates=False, date_parser=None, thousands=None, comment=None, skipfooter=0, convert_float=None, mangle_dupe_cols=True, storage_options=None)`

In [5]:
# Filepath
fp = r"\\szh.loc\ssz\applikationen\OGD\Daten\Quelldaten\SSZ\Politik\abstimmungen\__Modules_Abstimmungen_Export_AbstimmungenExport.xls"

# Read the data
imp_abstimmungsresultate = pd.read_excel(
    io = fp, 
    parse_dates=True,
    dtype={'Abstimmungs_Text': str}
    )



In [6]:
imp_abstimmungsresultate.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9798 entries, 0 to 9797
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Abstimmungs_Datum      9798 non-null   datetime64[ns]
 1   Nr_Politische_Ebene    9798 non-null   int64         
 2   Name_Politische_Ebene  9798 non-null   object        
 3   Abstimmungs_Text       9798 non-null   object        
 4   Nr_Resultat_Gebiet     9798 non-null   int64         
 5   Name_Resultat_Gebiet   9798 non-null   object        
 6   Nr_Wahlkreis_StZH      5215 non-null   float64       
 7   Name_Wahlkreis_StZH    5215 non-null   object        
 8   Stimmberechtigt        6662 non-null   float64       
 9   Ja                     9798 non-null   int64         
 10  Nein                   9798 non-null   int64         
 11  Stimmbeteiligung (%)   9798 non-null   float64       
 12  Ja (%)                 9798 non-null   float64       
 13  Nei

In [13]:
latest = imp_abstimmungsresultate.query("Abstimmungs_Datum == '2021-11-28'")
#latest.head(200)

## Gibt es beim Import Probleme?
Damit sauber exportiert werden kann, müssen die Daten auch sauber eingelesen werden. Wie können wir das aber testen?

Was sind die unique values per feature

In [14]:
imp_abstimmungsresultate.apply(lambda x: x.unique().size, axis=0)

Abstimmungs_Datum         416
Nr_Politische_Ebene         3
Name_Politische_Ebene       3
Abstimmungs_Text         2505
Nr_Resultat_Gebiet          3
Name_Resultat_Gebiet        3
Nr_Wahlkreis_StZH          17
Name_Wahlkreis_StZH        17
Stimmberechtigt          1183
Ja                       8627
Nein                     8246
Stimmbeteiligung (%)     1581
Ja (%)                    851
Nein (%)                  851
StaendeJa                  95
StaendeNein                89
dtype: int64

## Speichere die neuen Resultate in ein OGD-konformes CSV-File

OGD-konform bedeutet:
- **Datenformat**: `CSV`, 
- **Encoding**: in `UTF8`, 
- **Strings**: Zwischen Anführungszeichen `""` 
- **Separator**: ` , `

Link zu `to_csv()` in Pandas: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

DataFrame.to_csv(path_or_buf=None, sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression='infer', quoting=None, quotechar='"', line_terminator=None, chunksize=None, date_format=None, doublequote=True, escapechar=None, decimal='.', errors='strict', storage_options=None)

In [15]:
export_path=r"\\szh.loc\ssz\applikationen\OGD\Daten\Quelldaten\SSZ\Politik\abstimmungen\abstimmungen_seit1933.csv"

imp_abstimmungsresultate.to_csv(
    export_path,
    encoding="UTF-8-sig",
    quotechar='"',
    quoting=2,
    index=False,
    date_format='%Y-%m-%d'
)
print('done')

# encoding="UTF-8",
#date_format='%Y-%m-%dT%H:%M:%SZ'


done


## Testing

Teste den import der eben exportierten Daten. Klappt alles wie gewollt?

In [16]:
# Read the data
checkNewCsv = pd.read_csv(
    export_path, 
    parse_dates=True,
    )

In [19]:
#checkNewCsv.info(2)


In [20]:
from pivottablejs import pivot_ui

pivot_ui(checkNewCsv)

In [24]:
checkNewCsv.query("Abstimmungs_Datum == '2021-11-28' & Nr_Politische_Ebene==2")

,Abstimmungs_Datum,Nr_Politische_Ebene,Name_Politische_Ebene,Abstimmungs_Text,Nr_Resultat_Gebiet,Name_Resultat_Gebiet,Nr_Wahlkreis_StZH,Name_Wahlkreis_StZH,Stimmberechtigt,Ja,Nein,Stimmbeteiligung (%),Ja (%),Nein (%),StaendeJa,StaendeNein
39,2021-11-28,2,Kanton Zürich,Energiegesetz (EnerG) (Änderung vom 19. April ...,2,Kanton Zürich,NaN,NaN,924167.0,359275,214331,63.6,62.6,37.4,NaN,NaN
40,2021-11-28,2,Kanton Zürich,Energiegesetz (EnerG) (Änderung vom 19. April ...,3,Stadt Zürich,NaN,NaN,234112.0,106698,36598,62.7,74.5,25.5,NaN,NaN
41,2021-11-28,2,Kanton Zürich,Energiegesetz (EnerG) (Änderung vom 19. April ...,3,Stadt Zürich,2.0,Kreis 1+2,22808.0,10101,4004,63.4,71.6,28.4,NaN,NaN
42,2021-11-28,2,Kanton Zürich,Energiegesetz (EnerG) (Änderung vom 19. April ...,3,Stadt Zürich,3.0,Kreis 3,27882.0,13259,3653,62.2,78.4,21.6,NaN,NaN
43,2021-11-28,2,Kanton Zürich,Energiegesetz (EnerG) (Änderung vom 19. April ...,3,Stadt Zürich,4.0,Kreis 4+5,24457.0,12214,2309,60.9,84.1,15.9,NaN,NaN
44,2021-11-28,2,Kanton Zürich,Energiegesetz (EnerG) (Änderung vom 19. April ...,3,Stadt Zürich,5.0,Kreis 6,19552.0,10635,2820,70.5,79.0,21.0,NaN,NaN
45,2021-11-28,2,Kanton Zürich,Energiegesetz (EnerG) (Änderung vom 19. April ...,3,Stadt Zürich,6.0,Kreis 7+8,31373.0,15381,5933,69.6,72.2,27.8,NaN,NaN
46,2021-11-28,2,Kanton Zürich,Energiegesetz (EnerG) (Änderung vom 19. April ...,3,Stadt Zürich,7.0,Kreis 9,30973.0,12932,5289,60.2,71.0,29.0,NaN,NaN
47,2021-11-28,2,Kanton Zürich,Energiegesetz (EnerG) (Änderung vom 19. April ...,3,Stadt Zürich,8.0,Kreis 10,24277.0,12382,3998,69.0,75.6,24.4,NaN,NaN
48,2021-11-28,2,Kanton Zürich,Energiegesetz (EnerG) (Änderung vom 19. April ...,3,Stadt Zürich,9.0,Kreis 11,37043.0,14709,6029,57.4,70.9,29.1,NaN,NaN
